# Final Project: Challenge 8
Contributions From: Ryan Cottone

**Difficulty**: <span style="color:red">$\bullet \bullet \bullet$</span>

Welcome to the penultimate challenge!

You only have access to the public API for the server (`login` function). You do not have access to the encrypted log of the `admin`'s session, nor the password database.

The following functions has been changed:
- **hash**
- **compareHash**

The server will use `compareHash` and `hash` to evaluate user passwords instead of hash_SHA256. The code for these is below.

Recover the password for the user `admin`.



In [ ]:
def hash(data):
    if type(data) == int:
        data = intToText(data)
        
    finalStr = ""
    
    for i in range(min(30, len(data))):
        f = intToText(hash_SHA256(textToInt(data[i])))
        finalStr += f[0]

    
    finalStr = finalStr.ljust(30, '0')
    
    return textToInt(finalStr)

def compareHash(expectedHash, data):
    if type(data) == int:
        data = intToText(data)
    
    expectedStr = intToText(expectedHash)
    
    for i in range(len(data)):
        f = intToText(hash_SHA256(textToInt(data[i])))[0]

        if expectedStr[(i):((i+1))] != f:
            return False
        
        time.sleep(0.01) # Hopefully this will help prevent brute-force attacks by introducing a hash delay
    return True

In [ ]:
%run client.ipynb
%run server.ipynb
from utils import *

In [ ]:
from base64 import b64encode, b64decode
from Crypto.Cipher import AES
from Crypto.Hash import HMAC, SHA256
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
import time

In [ ]:
ellipticCurve = [0, 7]
ellipticCurveModulus = 115792089237316195423570985008687907853269984665640564039457584007908834671663 
generatorPoint = (55066263022277343669578718895168534326250603453777594175500187360389116729240,
 32670510020758816978083085130507043184471273380659243275938904335757337482424)
generatorPointOrder =115792089237316195423570985008687907852837564279074904382605163141518161494337

**As in previous challenges, you are given a function `login(username, password)` that makes the login request for you, and returns True if the login succeeds. You must only use this function to interact with the server.**

In [ ]:
def recoverPassword(username, login):
    ...

Here's an example to test with:

In [ ]:
random.seed(0) # DON'T CHANGE THIS 

def setup():
        # New client
    client = Client()

    # Client initiates server connection
    server = Server()
    server.acceptConnection(client.RSA_PUBLIC_KEY)

    server_PK, signature = server.publishRSAPublicKey()

    assert client.verifyServerCertificate(server_PK, signature)

    # Once verified, ask for ECDH parameters
    ellipticCurve, ellipticCurveModulus, generatorPoint, generatorPointOrder, signature = server.generateECDHParameters()

    assert client.verifyECDHParameters(ellipticCurve, ellipticCurveModulus, generatorPoint, generatorPointOrder, signature)

    # Once ECDH parameters are verified, generate our secret + public value
    client_ECDH = client.generateSignedECDHMessage()
    server_ECDH = server.generateSignedECDHMessage()

    # client gets server ecdh 
    client.acceptECDHMessage(server_ECDH[0], server_ECDH[1])

    # server gets client ecdh
    server.acceptECDHMessage(client_ECDH[0], client_ECDH[1])

    assert client.SHARED_SECRET_INT == server.SHARED_SECRET_INT

    return client, server

client, server = setup()
server.LOGINS = {}
    
signupRequest = {'data': '{"iv": "KrY/UGPSVX0Xh49GoJKr5Q==", "ciphertext": "AzXOs9H2JPXEyIoDHrCe4gu6hoXelObn0ELokIIDUU5KFPfZN3XIqUvnNpwwc6SsF610f1KNR7NjJriiwo58o/C+9Qo65Eu6/nZIGrnrXtk="}',
 'hmac': 'b8496bd449fe73c39a991f42b1a31be544c4ea9c0051802912d902eef10703d1'}
    
server.handleRequest(signupRequest)

def login(username, password):
    loginRequest = client.generateRequest({'type': 'verifyLogin', 'username': username, 'password': password})
    
    try:
        res = server.handleRequest(loginRequest)
        return res
    except Exception as e:
        return False
    
recoveredPwd = recoverPassword('admin', login)
print('Recovered password:', recoveredPwd)

### Submission

Once you've recovered the password, enter it into the Gradescope page [here](https://www.gradescope.com/courses/406561/assignments/2461059) and copy-paste your **recoverPassword** code in the same assignment. (Unfortunately our usual autograder does not play nice with this style of challenge)